In [165]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Reds9, Viridis256
from bokeh.models import Legend, LegendItem, DatetimeTickFormatter, LinearColorMapper
from bokeh.transform import linear_cmap

output_file('temp.html')

In [198]:
def plot_lines(data_file, output_file, depth_type, start_date, end_date, num_outliers):
    data = pd.read_csv(data_file)
    data = data.set_index(pd.date_range(start=start_date, end=end_date).date)
    data = data.transpose()
    data = data.set_index(pd.date_range(start=start_date, periods=24, freq='H').time)

    data_outputs = pd.read_csv(output_file).set_index(pd.date_range(start=start_date, end=end_date).date)

    data_outputs = data_outputs.transpose()

    data_final = pd.concat([data,data_outputs])

    data_final = data_final.sort_values(by=depth_type,axis=1)
    #data_final.loc['tmd',:]

    #color_list = list(data_final.loc["color",:])
    color_list = ["#fbb4b9"] * data_final.shape[1]
    color_list[len(color_list)-1] = "Blue"
    color_list[0:num_outliers] = ["Red"] * num_outliers
    alpha_list = [0.5] * data_final.shape[1]
    alpha_list[0:num_outliers] = [1] * num_outliers
    alpha_list[len(alpha_list)-1] = 1

    numlines = len(data.columns)

    p = figure(width=1000, height=800, x_axis_type="datetime") 
    p.multi_line(xs=[data.index.values]*numlines,
                 ys=[data[name].values for name in data.iloc[0:24]],
                 line_color=color_list,
                 line_width=5,
                 line_alpha=alpha_list)
    return p

In [199]:
plot_taxis_v2 = plot_lines('../data/taxis_v2.csv',"../outputs/taxis_v2_out.txt",'od','1/1/2016','31/12/2018', 50)
show(plot_taxis_v2)

In [200]:
plot_taxis_v1 = plot_lines('../data/taxis_v1.csv',"../outputs/taxis_v1_out.txt",'od','1/1/2018','31/12/2018', 20)
show(plot_taxis_v1)

In [168]:
data_outputs['td'].quantile([0.25,0.5,0.75])

0.25    0.013069
0.50    0.032895
0.75    0.074495
Name: td, dtype: float64

In [169]:
def get_color_from_depth(depth):
    '''
    if depth < 0.270425: return '#fef0d9'
    if depth < 0.353502: return '#fdcc8a'
    '''
    if depth < 0.074495: return '#fef0d9'
    else: return '#fdcc8a' 


data_outputs['color'] = data_outputs['td'].map(lambda x: get_color_from_depth(x))

In [10]:
data.iloc[0:24]

,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,...,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
00:00:00,25270,11265,18290,5749,6779,7185,8286,11397,21207,24816,...,11000,8733,4311,5300,3026,4763,6360,9082,9517,5877
01:00:00,28511,8549,15949,3580,4232,4397,5177,7559,18460,21287,...,8960,6944,2453,3622,1559,2712,4411,7022,7700,3875
02:00:00,25490,6644,13396,2304,2458,2740,3280,4955,14862,18182,...,7426,5642,1645,2174,938,1767,2848,5351,6396,2565
03:00:00,21277,5416,10146,1595,1642,1903,2123,3268,10869,13562,...,5054,4575,1185,1416,719,1288,2026,3977,4933,1806
04:00:00,15419,4038,7159,1999,1710,2006,2060,2795,6651,7895,...,3658,3596,1260,1206,846,1296,1666,2867,3424,1526
05:00:00,8102,2396,3384,3426,3622,3302,3488,3679,3191,3309,...,2207,1825,1523,835,1673,1882,2158,1752,1672,1680
06:00:00,6051,3271,3464,9684,10158,9561,10087,9472,4136,3581,...,3042,2414,3256,1515,3810,4481,4993,2394,2159,3305
07:00:00,5254,4229,4285,15934,18543,17779,18072,17665,6477,5267,...,4157,3358,4588,2027,5195,5751,6472,3269,2897,4393
08:00:00,4940,6171,6000,19263,22385,21130,21081,20882,9282,8211,...,6213,4760,6245,2534,6943,7976,8606,4716,4287,6125
09:00:00,6055,9274,9314,17283,19837,19661,19409,19803,14043,11423,...,9148,7538,8144,3677,8308,9317,10630,7261,6368,7036
